In [5]:
import os
# directory = r'D:\github\localt20\dataInHandNow\afterpreprocessed'
directory = '/app/dataInHandNow/afterpreprocessed'  # for docker

import polars as pl
matches = pl.read_csv(os.path.join(directory, 'matches.csv'))
deliveries = pl.read_parquet(os.path.join(directory, 'deliveries.parquet'))
matchPlayers = pl.read_csv(os.path.join(directory, 'Matchplayers.csv'))
playerStats = pl.read_csv(os.path.join(directory, 'playerStats.csv'))
teamStats = pl.read_csv(os.path.join(directory, 'teamStats.csv'))
matchPlayers

match_id,player,country,player_id
i64,str,str,str
1001349,"""A Zampa""","""Australia""","""14f96089"""
1001349,"""AJ Finch""","""Australia""","""b8d490fd"""
1001349,"""AJ Turner""","""Australia""","""ff1e12a0"""
1001349,"""AJ Tye""","""Australia""","""7c7d63a2"""
1001349,"""B Stanlake""","""Australia""","""6834d1f2"""
…,…,…,…
995469,"""SMSM Senanayake""","""Sri Lanka""","""4c4fa80b"""
995469,"""SS Pathirana""","""Sri Lanka""","""753c95b9"""
995469,"""TM Dilshan""","""Sri Lanka""","""5bdcdb72"""


In [8]:
import tensorflow as tf

# Rnn

In [9]:
from tensorflow.keras import layers, Model, Input

# RNN Model for Ball-by-Ball Data
def build_rnn_model(input_shape):
    inputs = Input(shape=input_shape)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(inputs)
    x = layers.Bidirectional(layers.LSTM(32))(x)
    model = Model(inputs, x, name="BallByBall_RNN")
    return model

# Ann for Player Stats

In [10]:
# ANN Model for Player Stats (with weight sharing for each player)
def build_player_ann_model(input_shape):
    inputs = Input(shape=input_shape)
    x = layers.Dense(64, activation='relu')(inputs)
    x = layers.Dense(32, activation='relu')(x)
    model = Model(inputs, x, name="Player_ANN")
    return model

# Ann for team Stats

In [11]:
# ANN Model for Team Stats
def build_team_ann_model(input_shape):
    inputs = Input(shape=input_shape)
    x = layers.Dense(64, activation='relu')(inputs)
    x = layers.Dense(32, activation='relu')(x)
    model = Model(inputs, x, name="Team_ANN")
    return model

# Combined

In [12]:
# Combined Model
def build_combined_model(ball_shape, player_shape, team_shape):
    # Ball-by-ball data input
    ball_input = Input(shape=ball_shape)
    ball_rnn_model = build_rnn_model(ball_shape)
    ball_features = ball_rnn_model(ball_input)

    # Player stats inputs (11 players per team, 22 players total)
    player_input = [Input(shape=player_shape) for _ in range(22)]
    player_ann_model = build_player_ann_model(player_shape)
    player_features = [player_ann_model(player) for player in player_input]
    combined_player_features = layers.concatenate(player_features)

    # Team stats input (2 teams per match)
    team_input = [Input(shape=team_shape) for _ in range(2)]
    team_ann_model = build_team_ann_model(team_shape)
    team_features = [team_ann_model(team) for team in team_input]
    combined_team_features = layers.concatenate(team_features)

    # Combine all features
    combined_features = layers.concatenate([ball_features, combined_player_features, combined_team_features])

    # Fully Connected Layers
    x = layers.Dense(128, activation='relu')(combined_features)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    
    # Output Layer
    output = layers.Dense(2, activation='softmax')(x)  # Binary classification output

    # Build Model
    model = Model(inputs=[ball_input] + player_input + team_input, outputs=output)
    return model

In [14]:
# Model Compilation
ball_shape = (None, 10)  # Example shape for ball-by-ball input (sequence length, features)
player_shape = (15,)     # Example shape for player stats input (features)
team_shape = (10,)       # Example shape for team stats input (features)

model = build_combined_model(ball_shape, player_shape, team_shape)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_30      │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_31      │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_32      │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_33      │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_34      │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_35      │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_36      │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_37      │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_38      │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_39      │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_40      │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_41      │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_42      │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_43      │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_44      │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_45      │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_46      │ (None, 15)        │          0 │ -               

 Total params: 200,514 (783.26 KB)

 Trainable params: 200,514 (783.26 KB)

 Non-trainable params: 0 (0.00 B)